In [213]:
from collections import Counter
import numpy as np
import csv 

def read_file(path): 
    data =[]
    with open(path) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            data.append(row)
    return data


        

class Node:
    def __init__(self, data):
        self.right = None
        self.left = None
        self.data = data
        self.depth = 0
        self.majority_vote = ""
        self.split_index = -1
        self.list_split_index = []

class Tree:
    def __init__(self, trainData, testData, Max_depth):
        self.header = trainData[0]
        self.root = Node(trainData[1:])
        self.test_data = testData[1:]
        
        self.label = []
        self.splitIndx = []
        
        self.StoreLabel(self.root.data)
        
        self.train_error = 0
        self.test_error = 0
        
        
        self.MAX_DEPTH = Max_depth
        self.leaves_nodes = []
        
        self.train_predict = []
        self.test_predict = []
        
        self.cal_attr_gini(self.root)
        
        self.evaluate('tr')
        self.evaluate('ts')
        
        self.calculateError(self.root.data, self.train_predict,'tr')
        self.calculateError(self.test_data, self.test_predict,'ts')
        
        
    def StoreLabel(self, data):
        self.label.append(data[0][-1])
        for i in range (1, len(data)):
            if data[i][-1] != data[0][-1]:
                self.label.append(data[i][-1])
                return
                       
            
        
    def prob (self, col, value):
        return col.count(value)/len(col)
    
    def splitData(self, data,index):
        A = data[0][index]
        lst1 =[]
        lst2 = []
        lst1.append(data[0])
        for i in range (1,len(data)):
            if data[i][index] == A:
                lst1.append(data[i])
            else:
                lst2.append(data[i])
        return(lst1,lst2)
                
    def majorityVote(self, node):
        datalist = node.data
        a = 0
        b = 0
        for j in range (0,len(datalist)):
            if datalist[j][-1] == self.label[0]:
                a +=1
            else:
                b +=1
        if a == b :
            if self.label[0] > self.label[1]:
                node.majority_vote = self.label[0]
            else:
                node.majority_vote = self.label[1]
        else:    
            if a > b:
                node.majority_vote = self.label[0]
            else:
                node.majority_vote = self.label[1]
            
    def cal_attr_gini(self,node):
        if (node.depth < self.MAX_DEPTH):
            data = node.data
            G_data = self.gini(data)
            G_attr = []
            for i in range (0,len(data[0])-1):
                splited_data = self.splitData(data,i)
                G_A = self.gini(splited_data[0])
                G_B = self.gini(splited_data[1])
                gini_gain = G_data - ((len(splited_data[0])/len(data)) * G_A) - ((len(splited_data[1])/len(data)) * G_B)
                G_attr.append(gini_gain)
            
            largest_gini = np.max(G_attr)
            list_argmax_ginis = [i for i, x in enumerate(G_attr) if x == largest_gini]
            if (largest_gini > 0):
                best_attr_index = np.max(list_argmax_ginis)
                left_right = self.best_attr_split(node,best_attr_index)
                node.left.split_index = best_attr_index
                node.right.split_index = best_attr_index
                for i in node.list_split_index:
                    node.left.list_split_index.append(i)
                    node.right.list_split_index.append(i)
                self.cal_attr_gini(left_right[0])
                self.cal_attr_gini(left_right[1])
            else:           
                self.majorityVote(node)
                self.leaves_nodes.append(node)
        else:
            self.majorityVote(node)
            self.leaves_nodes.append(node)

    def printTree(self):
        print(self.printedTree(self.root))
        
    def printedTree(self, node ,level=0):
        data = node.data
        collection = []
        for i in data:
            collection.append(i[-1])
        unique = np.unique(collection)
#         print (unique)
#         print(node.left.data)
        if node.split_index >=0:
            header = self.header[node.split_index] + " = " + node.data[0][node.split_index] + " : "
        else :
            header = ""

        strg = header + "[" + str(collection.count(self.label[0])) + " " + self.label[0] + " /"+ str(collection.count(self.label[1])) + " " + self.label[1] + "]"
        ret = "| "*level +strg+"\n"
        if node.left != None and node.right != None:
            ret += self.printedTree(node.left, level+1)
            ret += self.printedTree(node.right, level+1)
#         out = print(ret)
        return ret
        
    
    def calculateError(self, data_label, data_predict,char):    
        error = 0
        for i in range (0, len(data_label)):
            if data_label[i][-1] != data_predict[i]:
                error += 1
        error = error/len(data_label)
        if char == 'tr':
            self.train_error = error
        else:
            self.test_error = error
    
    def best_attr_split(self,node,indx):
        self.splitIndx.append(indx)
        node.list_split_index.append(indx)
        data = node.data
        best_splited_data = self.splitData(data,indx)
        node.left = Node(best_splited_data[0])
        node.right = Node(best_splited_data[1])
        node.left.depth = node.depth + 1
        node.right.depth = node.depth + 1 
        return (node.left,node.right)      
        
    def gini(self,lst):
        col = []
        prob_squr = 0
        for i in lst:
            col.append(i[-1])
        unique_value = np.unique(col)
        for i in unique_value:
            prob_squr = prob_squr - (self.prob(col,i))**2
        gini = 1 + prob_squr
        return gini
    
    def evaluate(self,char):
        if char == 'tr':
            data = self.root.data
        else:
            data = self.test_data
        predict = []
        for i in range (0, len(data)):
            for j in self.leaves_nodes:
                check = True
                indx_counter = 0
                while (check == True and indx_counter < len(j.list_split_index)):
                    if (data[i][j.list_split_index[indx_counter]] != j.data[0][j.list_split_index[indx_counter]]):
                        check = False
                        indx_counter = len(j.list_split_index)
                    else:
                        indx_counter += 1
                        
                if (check is True):
                    predict.append(j.majority_vote)
                    
        if char == 'tr':
            self.train_predict = predict 
        else:
            self.test_predict = predict[:]
    
class_ex = [['Y','A','B'],[1,0,0],[1,0,0],[1,0,1],[1,0,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]


small_data_train = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/small_train.tsv')
small_data_test = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/small_test.tsv')

education_data_train = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/education_train.tsv')
education_data_test = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/education_test.tsv')

politicians_train = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/politicians_train.tsv')
politicians_test = read_file('C:/Users/bnabi/Desktop/Master/Spring 2020/Machine Learning/HW02/handout/politicians_test.tsv')


politicians_tree = Tree(politicians_train,politicians_test, 3)
# print (politicians_tree.train_error)
# print (politicians_tree.test_error)
print(politicians_tree.header)
politicians_tree.printTree()
# lst = [[1,1,0,0],[1,1,2,1],[1,0,0,1],[1,1,2,1],[0,0,2,0],[0,1,1,0],[0,0,0,0]]
# small_data_train_tree = Tree(small_data_train, small_data_test, 3)

# print(small_data_train.gini(lst))


# print(small_data_train_tree.train_error)
# print(small_data_train_tree.test_error)

# print(small_data_train_tree.train_predict)
# print()
# print(small_data_train_tree.test_predict)
# print()

# small_data_train_tree.printTree()


# print('------------------------------------------------------')
# education_tree_train = Tree(education_data_train,education_data_test ,3)
# print (education_tree_train.train_error)
# print (education_tree_train.test_error)
# education_tree_train.printTree()


['Anti_satellite_test_ban', ' Aid_to_nicaraguan_contras', ' Mx_missile', ' Immigration', ' Superfund_right_to_sue', ' Duty_free_exports', ' Export_south_africa', ' Party ']
[83 democrat /66 republican]
|  Superfund_right_to_sue = y : [28 democrat /64 republican]
| |  Aid_to_nicaraguan_contras = n : [13 democrat /58 republican]
| | |  Duty_free_exports = y : [5 democrat /5 republican]
| | |  Duty_free_exports = n : [8 democrat /53 republican]
| |  Aid_to_nicaraguan_contras = y : [15 democrat /6 republican]
| | |  Mx_missile = y : [3 democrat /6 republican]
| | |  Mx_missile = n : [12 democrat /0 republican]
|  Superfund_right_to_sue = n : [55 democrat /2 republican]
| |  Export_south_africa = y : [55 democrat /1 republican]
| | |  Immigration = n : [46 democrat /0 republican]
| | |  Immigration = y : [9 democrat /1 republican]
| |  Export_south_africa = n : [0 democrat /1 republican]



In [ ]:
from collections import Counter
import numpy as np
import csv 

def read_file(path): 
    data =[]
    with open(path) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            data.append(row)
    return data


        

class Node:
    def __init__(self, data):
        self.right = None
        self.left = None
        self.data = data
        self.depth = 0
        self.majority_vote = ""
        self.split_index = -1
        self.list_split_index = []

class Tree:
    def __init__(self, trainData, testData, Max_depth):
        self.header = trainData[0]
        self.root = Node(trainData[1:])
        self.test_data = testData[1:]
        
        self.label = []
        self.splitIndx = []
        
        self.StoreLabel(self.root.data)
        
        self.train_error = 0
        self.test_error = 0
        
        
        self.MAX_DEPTH = Max_depth
        self.leaves_nodes = []
        
        self.train_predict = []
        self.test_predict = []
        
        self.cal_attr_gini(self.root)
        
        self.evaluate('tr')
        self.evaluate('ts')
        
        self.calculateError(self.root.data, self.train_predict,'tr')
#         self.calculateError(self.test_data, self.test_predict,'ts')
        
        
    def StoreLabel(self, data):
        self.label.append(data[0][-1])
        for i in range (1, len(data)):
            if data[i][-1] != data[0][-1]:
                self.label.append(data[i][-1])
                return
                       
            
        
    def prob (self, col, value):
        return col.count(value)/len(col)
    
    def splitData(self, data,index):
        A = data[0][index]
        lst1 =[]
        lst2 = []
        lst1.append(data[0])
        for i in range (1,len(data)):
            if data[i][index] == A:
                lst1.append(data[i])
            else:
                lst2.append(data[i])
        return(lst1,lst2)
                
    def majorityVote(self, node):
        datalist = node.data
        a = 0
        b = 0
        for j in range (0,len(datalist)):
            if datalist[j][-1] == self.label[0]:
                a +=1
            else:
                b +=1
        if a == b :
            if self.label[0] > self.label[1]:
                node.majority_vote = self.label[0]
            else:
                node.majority_vote = self.label[1]
        else:    
            if a > b:
                node.majority_vote = self.label[0]
            else:
                node.majority_vote = self.label[1]
            
    def cal_attr_gini(self,node):
        if (node.depth < self.MAX_DEPTH):
            data = node.data
            G_data = self.gini(data)
            G_attr = []
            for i in range (0,len(data[0])-1):
                splited_data = self.splitData(data,i)
                G_A = self.gini(splited_data[0])
                G_B = self.gini(splited_data[1])
                gini_gain = G_data - ((len(splited_data[0])/len(data)) * G_A) - ((len(splited_data[1])/len(data)) * G_B)
                G_attr.append(gini_gain)
                
            largest_gini = np.max(G_attr)
            list_argmax_ginis = [i for i, x in enumerate(G_attr) if x == largest_gini]
            if (largest_gini > 0):
                best_attr_index = np.max(list_argmax_ginis)
                print(best_attr_index)
                left_right = self.best_attr_split(node,best_attr_index)
                node.left.split_index = best_attr_index
                node.right.split_index = best_attr_index
                for i in node.list_split_index:
                    node.left.list_split_index.append(i)
                    node.right.list_split_index.append(i)
                self.cal_attr_gini(left_right[0])
                self.cal_attr_gini(left_right[1])
            else:           
                self.majorityVote(node)
                self.leaves_nodes.append(node)
        else:
            self.majorityVote(node)
            self.leaves_nodes.append(node)

    def printTree(self):
        print(self.printedTree(self.root))
        
    def printedTree(self, node ,level=0):
        data = node.data
        collection = []
        for i in data:
            collection.append(i[-1])
        unique = np.unique(collection)
#         print (unique)
#         print(node.left.data)
        if node.split_index >=0:
            header = self.header[node.split_index] + " = " + str(node.data[0][node.split_index]) + " : "
        else :
            header = ""

        strg = header + "[" + str(collection.count(self.label[0])) + " " + str(self.label[0]) + " /"+ str(collection.count(self.label[1])) + " " + str(self.label[1]) + "]"
        ret = "| "*level +strg+"\n"
        if node.left != None and node.right != None:
            ret += self.printedTree(node.left, level+1)
            ret += self.printedTree(node.right, level+1)
#         out = print(ret)
        return ret
        
    
    def calculateError(self, data_label, data_predict,char):    
        error = 0
        for i in range (0, len(data_label)):
            if data_label[i][-1] != data_predict[i]:
                error += 1
        error = error/len(data_label)
        if char == 'tr':
            self.train_error = error
        else:
            self.test_error = error
    
    def best_attr_split(self,node,indx):
        self.splitIndx.append(indx)
        node.list_split_index.append(indx)
        data = node.data
        best_splited_data = self.splitData(data,indx)
        node.left = Node(best_splited_data[0])
        node.right = Node(best_splited_data[1])
        node.left.depth = node.depth + 1
        node.right.depth = node.depth + 1 
        return (node.left,node.right)      
        
    def gini(self,lst):
        col = []
        prob_squr = 0
        for i in lst:
            col.append(i[-1])
        unique_value = np.unique(col)
        for i in unique_value:
            prob_squr = prob_squr - (self.prob(col,i))**2
        gini = 1 + prob_squr
        return gini
    
    def evaluate(self,char):
        if char == 'tr':
            data = self.root.data
        else:
            data = self.test_data
        predict = []
        for i in range (0, len(data)):
            for j in self.leaves_nodes:
                check = True
                indx_counter = 0
                while (check == True and indx_counter < len(j.list_split_index)):
                    if (data[i][j.list_split_index[indx_counter]] != j.data[0][j.list_split_index[indx_counter]]):
                        check = False
                        indx_counter = len(j.list_split_index)
                    else:
                        indx_counter += 1
                        
                if (check is True):
                    predict.append(j.majority_vote)
                    
        if char == 'tr':
            self.train_predict = predict 
        else:
            self.test_predict = predict[:]
    
class_ex = [['Y','A','B'],[1,0,0],[1,0,0],[1,0,1],[1,0,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]


lst = [['A','B','C','Y'],[1,1,0,0],[1,1,2,1],[1,0,0,1],[1,1,2,1],[0,0,2,0],[0,1,1,0],[0,0,0,0]]
lstA1 = [[1,1,0,0],[1,1,2,1],[1,0,0,1],[1,1,2,1]] 
lstA0 = [[0,0,2,0],[0,1,1,0],[0,0,0,0]]
lstB1 = [[1,1,0,0],[1,1,2,1],[1,1,2,1],[0,1,1,0]] 
lstB0 = [[1,0,0,1],[0,0,2,0],[0,0,0,0]] 
lstC0 = [[1,1,0,0],[1,0,0,1],[0,0,0,0]]
lstC1 = [[0,1,1,0]]
lstC2 = [[1,1,2,1],[1,1,2,1],[0,0,2,0]]
t = Tree(lst,[],4)
# t.cal_attr_gini(t.root)
t.printTree()

In [158]:

import numpy as np

class Node:
    def __init__(self, data):
        self.data = data
        self.majority_vote = ""
        self.attr = ""

class Tree:
    def __init__(self, user_data, testData, index):
        self.root = Node(user_data)
        self.right = Node([])
        self.left = Node([])
        self.testData = testData
        self.index = index
        
        self.splitData()


    
    def splitData(self):
        A = self.root.data[0][int(self.index)]
        self.right.data.append(self.root.data[0])
        for i in range (1,len(self.root.data)):
            if self.root.data[i][self.index] == A:
                self.right.data.append(self.root.data[i])
                self.right.attr = A
            else:
                self.left.data.append(self.root.data[i])
                self.left.attr = self.root.data[i][int(self.index)]
        
        self.majorityVote(self.right)
        self.majorityVote(self.left)
        
        
                
    def majorityVote(self, node):
        datalist = node.data
        checkA = datalist[0][-1]
        checkB = ""
        a = 0
        b = 0 
        for j in range (0,len(datalist)):
            if datalist[j][-1] == checkA:
                a +=1
            else:
                checkB = datalist[j][-1]
                b +=1
        if a == b :
            node.majority_vote = np.random.choice((checkA,checkB))
        else:    
            if a > b:
                node.majority_vote = checkA
            else:
                node.majority_vote = checkB
            
        
    def test_predict(self):
        test_pred =[]
        test_error = 0
        for i in range(0, len(self.testData)):
            if self.testData[i][self.index] == self.right.attr:
                test_pred.append(self.right.majority_vote)
                test_error += self.error(self.testData[i][-1],self.right.majority_vote)
            else:
                test_pred.append(self.left.majority_vote)
                test_error += self.error(self.testData[i][-1],self.left.majority_vote)
        test_error = test_error/len(self.testData)
        return (test_pred,test_error)
    
    def train_predict(self):
        train_pred =[]
        train_error = 0
        for i in range(0, len(self.root.data)):
            if self.root.data[i][self.index] == self.right.attr:
                train_pred.append(self.right.majority_vote)
                train_error += self.error(self.root.data[i][-1],self.right.majority_vote)
            else:
                train_pred.append(self.left.majority_vote)
                train_error += self.error(self.root.data[i][-1],self.left.majority_vote)
        train_error = train_error/len(self.root.data)
        return (train_pred,train_error)    

    def error(self, a, b):
        if a == b:
            return 0
        else:
            return 1
        

def read_file(path):
    data =[]
    with open('./'+path) as tsvfile:
        reader = csv.reader(tsvfile, delimiter='\t')
        for row in reader:
            data.append(row)
    return data [1:]

def write_file(path, predictions, mets=False):
    text = open('./'+path, 'w')
    if mets == True:
        text.write('error(train): '+str(predictions[0])+'\n'+'error(test): '+str(predictions[1]))
    else:
        text.write("\n".join(predictions))
    
    text.close()
    return text

train_data = read_file(train_in)
test_data = read_file(test_in)
indx = int(split_index)
t = Tree(train_data,test_data,indx)

write_file(train_out, t.train_predict()[0], mets = False)
write_file(test_out, t.test_predict()[0], mets = False)
write_file(metrics, [t.train_predict()[1], t.test_predict()[1]], mets = True)



SyntaxError: invalid syntax (<ipython-input-158-2f41f14ffade>, line 4)

In [155]:
[(i, j) for (i, j), x, val in enumerate(V) if x == np.max(V)]

ValueError: not enough values to unpack (expected 3, got 2)